## Deliverable 2. Create a Customer Travel Destinations Map.

In [26]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [27]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("/Users/asaleh/Desktop/Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Rikitea,PF,-23.1203,-134.9692,78.67,75,24,11.95,few clouds
1,Fuyu,CN,45.1833,124.8167,68.58,28,71,18.81,broken clouds
2,Ushuaia,AR,-54.8000,-68.3000,42.80,81,75,21.85,light intensity drizzle
3,Carnarvon,AU,-24.8667,113.6333,75.20,100,75,42.57,light rain
4,Buzdyak,RU,54.5833,54.5500,48.69,69,100,12.62,overcast clouds


In [30]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
Max_temp = float(input ("Enter the Maximum Temperature Criteria"))
Min_temp = float(input ("Enter the Minimum Temperature Criteria"))

Enter the Maximum Temperature Criteria90
Enter the Minimum Temperature Criteria70


In [31]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
city_data_df = city_data_df.loc[(city_data_df["Max Temp"] <= Max_temp) & (city_data_df["Max Temp"] >= Min_temp)]
city_data_df

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Rikitea,PF,-23.1203,-134.9692,78.67,75,24,11.95,few clouds
3,Carnarvon,AU,-24.8667,113.6333,75.20,100,75,42.57,light rain
7,Avarua,CK,-21.2078,-159.7750,78.80,94,97,1.14,overcast clouds
8,Atar,MR,20.5169,-13.0499,72.63,27,51,12.50,broken clouds
17,Pinotepa Nacional,MX,16.3167,-98.0167,71.89,77,3,3.11,clear sky
...,...,...,...,...,...,...,...,...,...
666,Padang,ID,-0.9492,100.3543,87.80,70,20,5.75,few clouds
673,Biak,ID,-0.9131,122.8766,83.70,69,83,1.48,broken clouds
675,Xichang,CN,27.8964,102.2634,86.00,6,7,12.95,clear sky
677,Iquitos,PE,-3.7481,-73.2472,77.00,88,75,5.75,moderate rain


In [32]:
# 4a. Determine if there are any empty rows.
city_data_df.isnull().count()

City                   234
Country                234
Lat                    234
Lng                    234
Max Temp               234
Humidity               234
Cloudiness             234
Wind Speed             234
Current Description    234
dtype: int64

In [33]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
city_data_clean_df = city_data_df.dropna()
city_data_clean_df.isnull().count()

City                   230
Country                230
Lat                    230
Lng                    230
Max Temp               230
Humidity               230
Cloudiness             230
Wind Speed             230
Current Description    230
dtype: int64

In [34]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = city_data_clean_df[["City", "Country", "Max Temp", "Lat", "Lng", "Current Description"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Current Description,Hotel Name
0,Rikitea,PF,78.67,-23.1203,-134.9692,few clouds,
3,Carnarvon,AU,75.20,-24.8667,113.6333,light rain,
7,Avarua,CK,78.80,-21.2078,-159.7750,overcast clouds,
8,Atar,MR,72.63,20.5169,-13.0499,broken clouds,
17,Pinotepa Nacional,MX,71.89,16.3167,-98.0167,clear sky,
18,Upata,VE,75.00,8.0086,-62.3989,overcast clouds,
20,Namatanai,PG,81.09,-3.6667,152.4333,moderate rain,
24,Hilo,US,71.60,19.7297,-155.0900,overcast clouds,
30,Itaituba,BR,72.19,-4.2761,-55.9836,light rain,
34,Cabo San Lucas,MX,70.00,22.8909,-109.9124,overcast clouds,


In [35]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    hotels
    #print(hotels)
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        hotel_df.loc[index, "Hotel Name"] = np.nan
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [36]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Lat,Lng,Current Description,Hotel Name
0,Rikitea,PF,78.67,-23.1203,-134.9692,few clouds,Pension Maro'i
3,Carnarvon,AU,75.20,-24.8667,113.6333,light rain,Hospitality Carnarvon
7,Avarua,CK,78.80,-21.2078,-159.7750,overcast clouds,Paradise Inn
8,Atar,MR,72.63,20.5169,-13.0499,broken clouds,Oumou Elghoura
17,Pinotepa Nacional,MX,71.89,16.3167,-98.0167,clear sky,Hotel Maria Del Carmen
...,...,...,...,...,...,...,...
666,Padang,ID,87.80,-0.9492,100.3543,few clouds,Mercure Padang
673,Biak,ID,83.70,-0.9131,122.8766,broken clouds,Penginapan Kim
675,Xichang,CN,86.00,27.8964,102.2634,clear sky,Xichang ibis Hotel
677,Iquitos,PE,77.00,-3.7481,-73.2472,moderate rain,Casa Morey Hotel & Restaurant


In [37]:
# 8a. Create the output File (CSV)
output_data_file = '/Users/asaleh/Desktop/Vacation_Search/WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file)

In [23]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [25]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))